# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [14]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint

# Import API key
from api_keys import geoapify_key

In [15]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tuktoyaktuk,69.4541,-133.0374,-7.60,84,100,10.36,CA,1674499026
1,1,half moon bay,37.4636,-122.4286,57.22,36,0,19.57,US,1674498775
2,2,suchil,23.6333,-103.9167,65.71,21,87,26.57,MX,1674499031
3,3,upernavik,72.7868,-56.1549,-4.63,95,100,10.09,GL,1674499032
4,4,san quintin,30.4833,-115.9500,58.28,32,0,9.95,MX,1674498953


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [16]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", "Lat", geo = True, size = "Humidity", scale = 1, color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Country"])

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [17]:
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tuktoyaktuk,69.4541,-133.0374,-7.60,84,100,10.36,CA,1674499026
1,1,half moon bay,37.4636,-122.4286,57.22,36,0,19.57,US,1674498775
2,2,suchil,23.6333,-103.9167,65.71,21,87,26.57,MX,1674499031
3,3,upernavik,72.7868,-56.1549,-4.63,95,100,10.09,GL,1674499032
4,4,san quintin,30.4833,-115.9500,58.28,32,0,9.95,MX,1674498953


In [18]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df=city_data_df.loc[(city_data_df["Humidity"]<=50) & (city_data_df["Max Temp"]<=80), :]
# Drop any rows with null values
ideal_cities=ideal_cities_df.dropna()
# Display sample data
ideal_cities.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,half moon bay,37.4636,-122.4286,57.22,36,0,19.57,US,1674498775
2,2,suchil,23.6333,-103.9167,65.71,21,87,26.57,MX,1674499031
4,4,san quintin,30.4833,-115.9500,58.28,32,0,9.95,MX,1674498953
24,24,ushuaia,-54.8000,-68.3000,62.26,48,75,6.91,AR,1674498833
35,35,jalu,29.0331,21.5482,57.00,28,0,9.91,LY,1674499073


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=ideal_cities.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,1,half moon bay,37.4636,-122.4286,57.22,36,0,19.57,US,1674498775,
2,2,suchil,23.6333,-103.9167,65.71,21,87,26.57,MX,1674499031,
4,4,san quintin,30.4833,-115.9500,58.28,32,0,9.95,MX,1674498953,
24,24,ushuaia,-54.8000,-68.3000,62.26,48,75,6.91,AR,1674498833,
35,35,jalu,29.0331,21.5482,57.00,28,0,9.91,LY,1674499073,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [28]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories":"accommodation.hotel","apiKey":geoapify_key, "limit":20}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude=row["Lat"]
    longitude=row["Lng"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    response = requests.get(base_url, params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    # Grab the first hotel from the results and store the details in the DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["address_line1"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No restaurant found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"nearest hotel in {hotel_df.loc[index, 'City']} : {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
nearest hotel in half moon bay : Half Moon Bay Inn
nearest hotel in suchil : No hotel found
nearest hotel in san quintin : Jardines Hotel
nearest hotel in ushuaia : Apart Hotel Aires del Beagle
nearest hotel in jalu : Galo Tourist Hotel
nearest hotel in raudeberg : No hotel found
nearest hotel in sunrise manor : Holiday Inn Express Las Vegas - Nellis
nearest hotel in broken hill : Ibis Styles
nearest hotel in mubi : No hotel found
nearest hotel in ginir : No hotel found
nearest hotel in meadow lake : No hotel found
nearest hotel in saint george : The Advenire, an Autograph Collection Hotel
nearest hotel in bloemfontein : Stadium Inn
nearest hotel in rosarito : WorldMark La Paloma
nearest hotel in gangoh : No hotel found
nearest hotel in tezu : No hotel found
nearest hotel in alice springs : Aurora Alice Springs
nearest hotel in porbandar : Toran Tourist Bungalow
nearest hotel in ridgecrest : Best Western China Lake Inn
nearest hotel in la seyne-sur-mer : Kyriad Pr

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,1,half moon bay,37.4636,-122.4286,57.22,36,0,19.57,US,1674498775,Half Moon Bay Inn
2,2,suchil,23.6333,-103.9167,65.71,21,87,26.57,MX,1674499031,No hotel found
4,4,san quintin,30.4833,-115.9500,58.28,32,0,9.95,MX,1674498953,Jardines Hotel
24,24,ushuaia,-54.8000,-68.3000,62.26,48,75,6.91,AR,1674498833,Apart Hotel Aires del Beagle
35,35,jalu,29.0331,21.5482,57.00,28,0,9.91,LY,1674499073,Galo Tourist Hotel
...,...,...,...,...,...,...,...,...,...,...,...
531,531,buraydah,26.3260,43.9750,63.05,49,31,10.40,SA,1674499715,دنا للوحدات السكنية
541,541,chifeng,42.2683,118.9636,-8.64,46,3,16.49,CN,1674499748,心连心酒店
543,543,bilma,18.6853,12.9164,74.48,11,0,6.64,NE,1674499750,No hotel found
546,546,iranshahr,27.2025,60.6848,52.14,22,0,13.80,IR,1674499754,Qasr Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [31]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", "Lat", geo = True, scale = 1, color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Country", "Hotel Name"])

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)